<a href="https://colab.research.google.com/github/Yi-Wei-Lin/Tibame_AI_Project/blob/main/Cabbage_predict_0707.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [309]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data = pd.read_excel("Cabbage.xlsx")

In [ ]:
data = data.dropna()
data = data.loc[data["Market"] == "台北一"]
np.array(data["Avg_price"]).reshape(-1,1)

In [ ]:
data_train = data[data["Date"] < "2019-01-01"]
data_test = data[data["Date"] >= "2019-01-01"]

In [ ]:
data_train = data_train.drop(["Date", "Market", "Product", "Month", "Week_day", "Year", "Rest_day"], axis=1)
data_train

In [ ]:
# from tensorflow.keras.utils import to_categorical
# pd.DataFrame(to_categorical(data_train["Month"]))

In [ ]:
Xscaler = MinMaxScaler()
Yscaler = MinMaxScaler()
ytrain = np.array(data_train["Avg_price"]).reshape(-1,1)
data_xtrain = Xscaler.fit_transform(data_train.drop(["Avg_price"],axis=1))
data_ytrain = Yscaler.fit_transform(ytrain)

In [ ]:
data_train = np.concatenate((data_xtrain, data_ytrain), axis=1)

In [ ]:
x_train = []
y_train = []
time_step = 60
for i in range(len(data_train) - time_step):
  x_train.append(data_train[i:(i+time_step)])
  y_train.append(data_train[(i+time_step)][-1])
x_train, y_train = np.array(x_train), np.array(y_train).reshape(-1,1)

In [ ]:
# Building LSTM model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
layers = [
    LSTM(units=12, activation="relu", return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])),
    Dropout(0.2),
    LSTM(units=24, activation="relu", return_sequences=True),
    Dropout(0.2),
    LSTM(units=48, activation="relu", return_sequences=True),
    Dropout(0.2),
    LSTM(units=96, activation="relu"),
    Dropout(0.2),
    Dense(units=1)
]
regressor = Sequential(layers)
regressor.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from keras.callbacks import EarlyStopping, ModelCheckpoint
regressor.compile(loss = 'mean_squared_error',
         optimizer = Adam()                   
)
callback = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("regressor.h5", save_best_only=True)
]
regressor.fit(x_train, y_train, epochs=20, batch_size=32, callbacks=callback)

In [ ]:
past_60_days = data[data["Date"] < "2019-01-01"].tail(time_step)
df = past_60_days.append(data_test, ignore_index = True)
df = df.drop(["Date", "Market", "Product", "Month", "Week_day", "Year", "Rest_day"], axis=1)
df

In [ ]:
real_avg_price = np.array(df.iloc[time_step:]["Avg_price"]).reshape(-1,1)
ytest = np.array(df["Avg_price"]).reshape(-1,1)
x_inputs = Xscaler.transform(df.drop(["Avg_price"], axis=1))
y_inputs = Yscaler.transform(ytest)

In [ ]:
inputs = np.concatenate((x_inputs, y_inputs), axis=1)

In [ ]:
x_test = []
y_test = []
time_step = 60
for i in range(len(inputs) - time_step):
  x_test.append(inputs[i:(i+time_step)])
  y_test.append(inputs[(i+time_step)][-1])
yy = y_test
x_test, y_test = np.array(x_test), np.array(y_test).reshape(-1,1)

In [ ]:
pre_price = regressor.predict(x_test)
pre_price = Yscaler.inverse_transform(pre_price)

In [ ]:
# Visualising the results
plt.figure(figsize=(14,5))
plt.plot(real_avg_price, color = 'red', label = 'Real Cabbage Price')
plt.plot(pre_price, color = 'blue', label = 'Predicted Cabbage Price')
plt.title('Cabbage Price Prediction')
plt.xlabel('Time')
plt.ylabel('Cabbage Price')
plt.legend()
plt.show()